# Constants

In [1]:
import os, json
from IPython.display import Markdown, display
from dotenv import load_dotenv # requires python-dotenv
from PIL import Image # requires pip install pillow
from datetime import datetime
from common.agents_helper_functions_NEW import *
import importlib.metadata
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

if not load_dotenv("./../config/credentials_my.env"):
    print("Environment variables not loaded, cell execution stopped")
else:
    print("Environment variables have been loaded ;-)")


project_endpoint = os.environ["AZURE_AIF_PROJECT_ENDPOINT"]
deployment_name =  os.environ["MODEL_DEPLOYMENT_NAME"]
api_version = os.environ["OPENAI_API_VERSION"] # at least "2025-03-01-preview"

print(f'Project Endpoint: <{project_endpoint}>')
print(f"azure-ai-projects library installed version: {importlib.metadata.version("azure-ai-projects")}")
print(f"azure-ai-agents library installed version: {importlib.metadata.version("azure-ai-agents")}")

Environment variables have been loaded ;-)
Project Endpoint: <https://aif01ais1lrx7.services.ai.azure.com/api/projects/aif01project01lrx7>
azure-ai-projects library installed version: 1.0.0b12
azure-ai-agents library installed version: 1.0.2


# [Create AI Foundry Agent Client](https://learn.microsoft.com/en-us/python/api/overview/azure/ai-agents-readme?view=azure-python-preview)
**Note**: I could create the `project` client rather than the `agent` client, however this is easier to read.<br/>
Please consider that `project_client.agens == agents_client`

In [2]:
from azure.ai.agents import AgentsClient
from azure.identity import DefaultAzureCredential

agents_client = AgentsClient(
    endpoint=project_endpoint,
    credential=DefaultAzureCredential(),
)

agents_client

# Check if we have any files already on Azure AI Foundry
For better clarity, we delete all pre-existing files, if present

In [3]:
# delete all files

i=0
for file in agents_client.files.list()["data"]:
    i += 1
    agents_client.files.delete(file_id=file.id)
    print(f"{i} - File <{file.filename}> ({file.id}) has been deleted")

print(f"Files deleted: {i}\n")

Files deleted: 0



# Create the [CodeInterpreterTool](https://learn.microsoft.com/en-us/azure/ai-services/agents/how-to/tools/code-interpreter-samples?pivots=python#code-interpreter-setup)

In [4]:
from azure.ai.agents.models import CodeInterpreterTool

code_interpreter = CodeInterpreterTool()
print(f"Code interpreter definitions: {code_interpreter.definitions}")
print(f"Code interpreter resources: {code_interpreter.resources}")

Code interpreter definitions: [{'type': 'code_interpreter'}]
Code interpreter resources: {}


# Upload file(s) to work with

# Create a MessageAttachment

In [5]:
from azure.ai.agents.models import FilePurpose

file = agents_client.files.upload_and_poll(file_path="./turbines.csv", purpose=FilePurpose.AGENTS)
print(f"Uploaded file: {file}")

Uploaded file: {'object': 'file', 'id': 'assistant-7pfgjyJuKPxXkb1iJazhG7', 'purpose': 'assistants', 'filename': 'turbines.csv', 'bytes': 576, 'created_at': 1751448965, 'expires_at': None, 'status': 'processed', 'status_details': None}


In [6]:
# Create an attachment
from azure.ai.agents.models import MessageAttachment

attachment = MessageAttachment(file_id=file.id, tools=code_interpreter.definitions)

attachment

{'file_id': 'assistant-7pfgjyJuKPxXkb1iJazhG7', 'tools': [{'type': 'code_interpreter'}]}

# Create AI Foundry Agent

In [7]:
# using project_client.agents...
ai_agent = agents_client.create_agent(
    model=os.environ["MODEL_DEPLOYMENT_NAME"],
    name="my-agent-with-messageattachment",
    instructions="You are helpful agent",
    tools=code_interpreter.definitions,
    tool_resources=code_interpreter.resources,
)

print(f"Created agent, agent ID: {ai_agent.id},\nagent items: {ai_agent.items}")

Created agent, agent ID: asst_CmoNbfXe5Creo4N5oRTCqvB8,
agent items: <bound method _MyMutableMapping.items of {'id': 'asst_CmoNbfXe5Creo4N5oRTCqvB8', 'object': 'assistant', 'created_at': 1751448965, 'name': 'my-agent-with-messageattachment', 'description': None, 'model': 'gpt-4o', 'instructions': 'You are helpful agent', 'tools': [{'type': 'code_interpreter'}], 'top_p': 1.0, 'temperature': 1.0, 'tool_resources': {'code_interpreter': {'file_ids': []}}, 'metadata': {}, 'response_format': 'auto'}>


# Create the thread and attach a new message to it **which includes the attachment**

In [8]:
from azure.ai.agents.models import MessageRole

# Create a thread
thread = agents_client.threads.create()
print(f"Created thread: {thread}\n")

message = agents_client.messages.create(
    thread_id=thread.id,
    role=MessageRole.USER,
    content="Hello, what's the maximum wind speed for my turbines?",
    attachments=[attachment]
)

Created thread: {'id': 'thread_KZg1kDCQHoGCR99xqqzIEZDX', 'object': 'thread', 'created_at': 1751448966, 'metadata': {}, 'tool_resources': {}}



# Run the agent syncrhonously

In [9]:
%%time

# Run the agent
run = agents_client.runs.create_and_process\
    (thread_id=thread.id, agent_id=ai_agent.id)

print(f"Run finished with status: {run.status}.\n\nRun: {run}")

if run.status == "failed":
    # Check if you got "Rate limit is exceeded.", then you want to get more quota
    print(f"Run failed: {run.last_error}")

Run finished with status: RunStatus.COMPLETED.

Run: {'id': 'run_EJ3MHkDMlpxfjy4rvrZmLYqj', 'object': 'thread.run', 'created_at': 1751448967, 'assistant_id': 'asst_CmoNbfXe5Creo4N5oRTCqvB8', 'thread_id': 'thread_KZg1kDCQHoGCR99xqqzIEZDX', 'status': 'completed', 'started_at': 1751448968, 'expires_at': None, 'cancelled_at': None, 'failed_at': None, 'completed_at': 1751448983, 'required_action': None, 'last_error': None, 'model': 'gpt-4o', 'instructions': 'You are helpful agent', 'tools': [{'type': 'code_interpreter'}], 'tool_resources': {}, 'metadata': {}, 'temperature': 1.0, 'top_p': 1.0, 'max_completion_tokens': None, 'max_prompt_tokens': None, 'truncation_strategy': {'type': 'auto', 'last_messages': None}, 'incomplete_details': None, 'usage': {'prompt_tokens': 24575, 'completion_tokens': 616, 'total_tokens': 25191, 'prompt_token_details': {'cached_tokens': 0}}, 'response_format': 'auto', 'tool_choice': 'auto', 'parallel_tool_calls': True}
CPU times: user 36.8 ms, sys: 3.07 ms, total: 

# Fetch messages from the thread after the agent run execution

In [10]:
from azure.ai.agents.models import MessageTextContent, MessageImageFileContent, MessageTextFileCitationAnnotation, MessageTextFilePathAnnotation

image_files = []
annotations = []
citations = []

if run.status == 'completed':
    messages = agents_client.messages.list(thread_id=thread.id)
    messages_list = list(agents_client.messages.list(thread_id=thread.id))  # Convert iterator to a list
    messages_nr = len(messages_list)
    print(f"Here are the {messages_nr} messages:\n")
    
    for i, message in enumerate(reversed(messages_list), 1):
        j = 0
        print(f"\n===== MESSAGE {i} =====")
        for c in message.content:
            j +=1
            if (type(c) is MessageTextContent):
                print(f"\nMessage {i} / CONTENT {j} (MessageTextContent) --> Text: {c.text.value}")
                for a in c.text.annotations:
                    if type(a) is MessageTextFileCitationAnnotation:
                        print(f">>> Citation in MessageTextContent {j} of message {i}: {a}\n")
                        citations.append(a)
                    elif type(a) is MessageTextFilePathAnnotation:
                        print(f">>> Annotation in MessageTextContent {j} of message {i}: {a}\n")
                        annotations.append(a)
            elif (type(c) is MessageImageFileContent):
                print(f"\nMessage {i} / CONTENT {j} (MessageImageFileContent) --> image_file id: {c.image_file.file_id}")
                image_files.append(c.image_file.file_id)

else:
    print(f"Sorry, I can't proceed because the run status is {run.status}")

Here are the 8 messages:


===== MESSAGE 1 =====

Message 1 / CONTENT 1 (MessageTextContent) --> Text: Hello, what's the maximum wind speed for my turbines?

===== MESSAGE 2 =====

Message 2 / CONTENT 1 (MessageTextContent) --> Text: Let's start by examining the contents of the file you uploaded to determine what information it contains regarding wind speeds for your turbines. I'll load the file and check its contents.

===== MESSAGE 3 =====

Message 3 / CONTENT 1 (MessageTextContent) --> Text: It seems there was an issue with recognizing the format of the file. Let's try explicitly specifying the engine to handle the file format and attempt to load it again.

===== MESSAGE 4 =====

Message 4 / CONTENT 1 (MessageTextContent) --> Text: It seems there was an issue identifying the file as an Excel file. It could be that the file is not in the correct format or it needs different handling.

Let's try reading the file again but with a more flexible approach to identify its format. If this f

# Retrieve and download eventual annotations

In [11]:
print (f"Nr. of file path annotations: {len(annotations)}\n")

i=0
for a in annotations:
    i += 1
    print(f"{i} - File annotation paths: {a.text}")
    file_name = a.text.split('/')[-1]
    file_id = a.file_path.file_id

    #agents_client.files.save(file_id=file_id, file_name=file_name)
    agents_client.files.save(file_id=file_id, file_name=file_name)
    print(f"\n>>> file <{file_id}> saved as <{file_name}>")
    
    # project_client.agents.save_file(file_id=file_path_annotation.file_path.file_id, file_name=file_name)
    print(f"File annotation {i} saved as file to: {os.getcwd()}/{file_name}")
    image = mpimg.imread(f"{os.getcwd()}/{file_name}") # read the image
    plt.imshow(image)
    plt.show()

Nr. of file path annotations: 0



# Retrieve eventual citations

In [12]:
print (f"Nr. of file path citations: {len(citations)}\n")

i=0
for a in citations:
    i += 1
    print(f"{i} - citation: {a}")

Nr. of file path citations: 0



# Retrieve and download eventual images

In [13]:
print (f"Nr. of image contents: {len(image_files)}\n")

i=0
# Generate an image file for the bar chart
for image_file in image_files:
    i += 1
    print(f"{i} - Image file id: {image_file}")
    file_name = f"{image_file}.png"
    agents_client.files.save(file_id=image_file, file_name=file_name)
    print(f"Image content {i} file to: {os.getcwd()}/{file_name}")
    image = mpimg.imread(f"{os.getcwd()}/{file_name}") # read the image
    plt.imshow(image)
    plt.show()

Nr. of image contents: 0



# Collect all resources for this project

In [14]:
all_agents = list_all_agents(client=agents_client)
print(all_agents["summary"])

all_threads = list_all_threads(client=agents_client)
print(all_threads["summary"])

all_files = list_all_files(client=agents_client)
print(all_files["summary"])

all_runs = list_all_runs(client=agents_client)
print(all_runs["summary"])

# all_runsteps=list_all_runsteps(project_client)
# print(all_runsteps["summary"])

# all_messages = list_all_messages(project_client)
# print(all_messages["summary"])

all_vectorstores = list_all_vectorstores(client=agents_client)
print(all_vectorstores["summary"])

1 agents
1 threads
1 files
1 runs in 1 threads
0 vector stores


# Teardown for all resources

In [15]:
# delete all vector stores

i=0
for vector_store in all_vectorstores["content"]:
    i += 1
    agents_client.vector_stores.delete(vector_store_id=vector_store.id)
    print(f"{i} - Vector store <{vector_store.id}> has been deleted")

all_vectorstores = list_all_vectorstores(client=agents_client)

print(f"Vector stores deleted: {i}\n")

Vector stores deleted: 0



In [16]:
# delete all files

i=0
for file in all_files['content']:
    i += 1
    agents_client.files.delete(file_id=file.id)
    print(f"{i} - File <{file.filename}> ({file.id}) has been deleted")

all_files = list_all_files(client=agents_client)

print(f"Files deleted: {i}\n")

1 - File <turbines.csv> (assistant-7pfgjyJuKPxXkb1iJazhG7) has been deleted
Files deleted: 1



In [17]:
# delete all threads

i=0
for thread in all_threads["content"]:
    i += 1
    agents_client.threads.delete(thread_id=thread.id)
    print(f"{i} - Thread <{thread.id}> has been deleted")

all_threads = list_all_threads(client=agents_client)

print(f"Threads deleted: {i}\n")

1 - Thread <thread_KZg1kDCQHoGCR99xqqzIEZDX> has been deleted
Threads deleted: 1



In [18]:
# delete all agents

i=0
for agent in all_agents["content"]:
    i += 1
    agents_client.delete_agent(agent_id=agent.id)
    print(f"{i} - Agent <{agent.id}> has been deleted")

all_agents = list_all_agents(client=agents_client)

print(f"Agents deleted: {i}\n")

1 - Agent <asst_CmoNbfXe5Creo4N5oRTCqvB8> has been deleted
Agents deleted: 1



# HIC SUNT LEONES